In [1]:
from gssutils import *

scraper = Scraper('https://www.gov.uk/government/statistics/immigration-statistics-october-to-december-2017-data-tables')
scraper

## Immigration statistics, October to December 2017: data tables second edition

Listing of the data tables included in 'Immigration statistics, October to December 2017’.

### Description

Listing of the data tables included in [‘Immigration Statistics, October to
December 2017’](https://www.gov.uk/government/statistics/immigration-
statistics-october-to-december-2017).

Data is in yearly or quarterly divisions.

To view and use ‘ODS’ files, OS X users can [download
OpenOffice](http://www.openoffice.org/download/).



### Distributions

1. Sponsorship data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/683358/sponsorship-oct-dec-2017-tables.ods))
1. Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 1 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/683359/entry-visas1-oct-dec-2017-tables.ods))
1. Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 2 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681238/entry-visas2-oct-dec-2017-tables.ods))
1. Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 3 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681241/entry-visas3-oct-dec-2017-tables.ods))
1. Admissions data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681242/admissions-oct-dec-2017-tables.ods))
1. Extensions of stay data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681243/extensions-oct-dec-2017-tables.ods))
1. Settlement data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681244/settlement-oct-dec-2017-tables.ods))
1. Citizenship data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/683360/citizenship-oct-dec-2017-tables.ods))
1. Asylum data tables immigration statistics October to December 2017 volume 1 second edition ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/691963/asylum1-oct-dec-2017-tables.ods))
1. Asylum data tables immigration statistics October to December 2017 volume 2 second edition ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/691964/asylum2-oct-dec-2017-tables.ods))
1. Asylum data tables immigration statistics October to December 2017 volume 3 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/691965/asylum3-oct-dec-2017-tables.ods))
1. Asylum data tables immigration statistics October to December 2017 volume 4 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/691967/asylum4-oct-dec-2017-tables.ods))
1. Asylum data tables immigration statistics October to December 2017 volume 5 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/691969/asylum5-oct-dec-2017-tables.ods))
1. Returns data tables immigration statistics October to December 2017 volume 1 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681246/returns1-oct-dec-2017-tables.ods))
1. Returns data tables immigration statistics October to December 2017 volume 2 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/683361/returns2-oct-dec-2017-tables.ods))
1. Returns data tables immigration statistics October to December 2017 volume 3 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/683362/returns3-oct-dec-2017-tables.ods))
1. Returns data tables immigration statistics October to December 2017 volume 4 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681250/returns4-oct-dec-2017-tables.ods))
1. Returns data tables immigration statistics October to December 2017 volume 5 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681251/returns5-oct-dec-2017-tables.ods))
1. Detention data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681703/detention-oct-dec-2017-tables.ods))
1. European Economic Area data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681253/eea-oct-dec-2017-tables.ods))


In [2]:
dist = scraper.distribution(
    title='Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 1'
)
book = dist.as_pandas()
sheet = book['vi_05']
%run "Entry clearance visas granted by country of nationality(Vi_05).ipynb"
entry_1 = tidy.copy()

sheet = book['vi_01_q']
%run "Entry clearance visa applications and resolution by category ( vi_01_q).ipynb"
entry_2 = tidy.copy()
entry_1.rename(columns={'Year': 'Period'}, inplace=True)
entry_1['Period'] = entry_1['Period'].astype(str)
entry_1.replace(
    {'Period': {r'([0-9]{4})': r'year/\1'}}, regex=True, inplace=True
)
for col in ['Application type', 'Applicant type', 'Application category', 'Resolution']:
    entry_1[col] = 'All'
# Todo: "World" or "Not UK"?
entry_2['Country of nationality'] = 'Rest of world'
combined = pd.concat([entry_1, entry_2], sort=True)
combined['Measure Type'] = 'Count'
combined['Unit'] = 'Applications'

In [3]:
from pathlib import Path
out = Path('out')
out.mkdir(exist_ok=True)

combined.to_csv(out / 'entry_visas.csv', index=False)

scraper.dataset.family = 'migration'

with open(out / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())